#### Theta-semantic distance correlations

In [ ]:
from gensim.models import KeyedVectors
model_full = KeyedVectors.load_word2vec_format('/home1/esolo/word2vec/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import zscore
import sys
sys.path.insert(0, '/home1/esolo/notebooks/codebases/')
import CML_stim_pipeline
from loc_toolbox import get_region, update_pairs
from semantic_toolbox import remove_repeats, get_recall_clustering, find_temporal_runs
import pickle as pk

#CML data
from cmlreaders import CMLReader, get_data_index
df = get_data_index("r1")

#Get all subjects who did FR1
FR_subs = df[df['experiment']=='FR1']['subject'].unique()

def justfinites(arr):
    return np.array(arr)[np.isfinite(arr)]

def get_elecs_and_evs(reader, ref, ap_thresh=None):
    evs = reader.load("events")
    contacts = reader.load("contacts")
    #pairs = reader.load("pairs")
    cmlpipe = CML_stim_pipeline.cml_pipeline(s, exp) #use this for loading electrodes with bad-elecs info
    cmlpipe.set_elecs(type=ref)
    pairs = cmlpipe.elecs
    try:
        locali = reader.load('localization')
        pairs = update_pairs(locali, pairs)                
    except:
        pass

    mtl_elecs = get_region(pairs, region='hipp', side=None)
    #mtl_elecs = pairs[pairs['ind.region'].str.contains('temporal').astype(bool) & (pairs['ind.x']!=0)]
    if 'electrode_categories' in mtl_elecs.keys():  #some subjects dont have bad electrode info
        mtl_elecs = mtl_elecs[~mtl_elecs['electrode_categories'].str.contains('bad')] #filter out bad electrodes
        
        #optionally, filter out epileptic electrodes
        mtl_elecs = mtl_elecs[~mtl_elecs['electrode_categories'].str.contains('ictal')]
        mtl_elecs = mtl_elecs[~mtl_elecs['electrode_categories'].str.contains('soz')]
        
    if ap_thresh is not None:
        mtl_elecs = mtl_elecs[mtl_elecs['avg.y']>ap_thresh]

    return mtl_elecs, evs

def get_list_words(word_evs, listnum):
    list_dat = word_evs[word_evs['list']==listnum]
    words = np.array(list_dat['item_name'])
    if 'AXE' in words:
        words[words=='AXE']='AX'  #seems to not have this spelling of ax
        
    return words, list_dat

def get_session_model(model, words, ndim):
    
    #Get PCA dims for *session-level* wordpool
    words = np.array(words)
    if 'AXE' in words:
        words[words=='AXE']='AX'  #seems to not have this spelling of ax
    word_mat = np.array([model[w.lower()] for w in words])
    pca = PCA(n_components=ndim)
    pcs = pca.fit_transform(word_mat)
    exp_var = pca.explained_variance_ratio_
    new_model = {}
    for idx, w in enumerate(words):
        new_model[w] = pcs[idx, :]
        
    return new_model, exp_var

def get_session_PCs(word_evs, new_model):
    
    list_dat = word_evs[word_evs['list']==listnum]
    list_words = np.array(list_dat['item_name'])
    if 'AXE' in list_words:
        list_words[list_words=='AXE']='AX'  #seems to not have this spelling of ax

    #Get semantic positions from new_model
    pcs = np.array([new_model[w_] for w_ in list_words])
    
    return pcs

def regress_output_pos(all_theta, all_trans, all_outputs, all_serial_pos):
    
    #Regress out output position
    import statsmodels.formula.api as sm

    #Set up feature matrices
    theta_feats = np.array(all_theta)[np.isfinite(all_trans)]
    output_position = np.array(all_outputs)[np.isfinite(all_trans)]
    all_serial_pos = np.array(all_serial_pos)[np.isfinite(all_trans)]
    trans_feats = justfinites(all_trans)
    X = np.empty([len(theta_feats), 4])
    X[:, 0] = trans_feats
    X[:, 1] = output_position
    X[:, 2] = all_serial_pos
    X[:, 3] = np.ones(len(theta_feats))
    y = theta_feats

    #Fit the model
    result = sm.OLS(y, X).fit()
    tru_coefs = copy(result.params)
    
    return tru_coefs

def residualize_theta(all_theta, all_outputs, all_serial_pos):
    
    #Regress out output position
    import statsmodels.formula.api as sm

    #Set up feature matrices
    theta_feats = np.array(all_theta)
    output_position = np.array(all_outputs)
    X = np.empty([len(theta_feats), 3])
    X[:, 0] = output_position
    X[:, 1] = np.array(all_serial_pos)
    X[:, 2] = np.ones(len(theta_feats))
    y = theta_feats

    #Fit the model
    result = sm.OLS(y, X).fit()
    y_pred = result.predict(X)
        
    return all_theta-y_pred

def residualize_con_by_power(con, power):
    
    import statsmodels.formula.api as sm
    
    #Set up feature matrices
    con_feats = np.array(con)
    pow_feats = np.array(power)
    X = np.empty([len(con_feats), 2])
    X[:, 0] = pow_feats
    X[:, 1] = np.ones(len(con_feats))
    y = con_feats
    
    #Fit the model
    result = sm.OLS(y, X).fit()
    y_pred = result.predict(X)
    
    return con_feats-y_pred
    
def get_power(reader, evs, mtl_elecs, freqs, ref):
    
    eeg = reader.load_eeg(events=evs, rel_start=-1000, rel_stop=0, scheme=mtl_elecs)  #Looking at 1 second prior to retrieval events

    #Now get MNE format for power extraction
    eeg = eeg.to_mne()
    if ref=='mono':
        eeg.set_eeg_reference(ref_channels=None) #Set to average reference
        eeg.apply_proj()
    eeg = eeg.resample(500)  #downsample to ___ Hz

    #Use MNE to get multitaper power in theta and HFA bands
    from mne.time_frequency import psd_multitaper
    theta_pow, fdone = psd_multitaper(eeg, fmin=freqs[0], fmax=freqs[1], tmin=0.0, verbose=False)  #power is (events, elecs, freqs)
    theta_pow = np.mean(np.log10(theta_pow), 1)  #Average across electrodes
    
    return theta_pow, fdone

def semantic_temporal_model(sem_feats, temp_feats, theta_feats):

    #Linear combination of semantic/temporal distances to predict theta power
    import statsmodels.formula.api as sm
    
    #Set up feature matrices
    X = np.empty([len(sem_feats), 3])
    X[:, 0] = sem_feats
    X[:, 1] = temp_feats
    X[:, 2] = np.ones(len(sem_feats))
    y = theta_feats
    
    #Fit the model
    result = sm.OLS(y, X).fit()
    
    return result

def get_long_short(arr, long_thresh=0.25, short_thresh=0.75):
    
    arr = np.array(arr)
    longs = arr<long_thresh
    num_longs = np.sum(longs)
    
    if num_longs<1:
        raise ValueError('Not enough long transitions!')
        
    #srt_arr = np.argsort(arr)
    #shorts = np.zeros(len(longs)).astype(bool)
    #shorts[srt_arr[-1*num_longs:]] = True
    
    shorts = arr>short_thresh
    
    #print('Longs:'+str(np.sum(longs)))
    #print('Shorts:'+str(np.sum(shorts)))
    
    return longs, shorts

def get_hipp_phg_conn(reader, evs, mtl_elecs, freqs, ref):
    
    eeg = reader.load_eeg(events=evs, rel_start=-1000, rel_stop=0, scheme=mtl_elecs)  #Looking at 1 second prior to retrieval events

    #Now get MNE format for power extraction
    eeg = eeg.to_mne()
    if ref=='mono':
        eeg.set_eeg_reference(ref_channels=None) #Set to average reference
        eeg.apply_proj()
    sr = 500
    eeg = eeg.resample(sr)  #downsample to ___ Hz
    
    #select only hippocampal elecs
    hipp_filt = np.zeros(len(mtl_elecs))
    for idx, i in enumerate(mtl_elecs['stein.region']):
        if np.sum([(s_ in i) for s_ in ['CA', 'DG', 'SUB', 'Sub']])>0:
            hipp_filt[idx] = 1
    hipp_filt = hipp_filt.astype(bool)
    if (np.sum(hipp_filt)==0) | np.sum(hipp_filt)==len(hipp_filt):  #don't run if no hipp/phg contacts
        raise ValueError

    #Use MNE to get multitaper power in theta and HFA bands
    from mne.connectivity import spectral_connectivity
    from scipy.special import logit
    
    #loop through events and get hipp/phg coherence
    event_coh = []
    for ev in range(len(eeg)):
        con, fdone, times, n_epochs, n_tapers = spectral_connectivity(eeg[ev], method='coh', mode='multitaper', sfreq=sr, fmin=freqs[0], fmax=freqs[1], faverage=False, tmin=0.0, 
                                                                          mt_adaptive=False, verbose=False, mt_bandwidth=None)
        #con = con[:, :, 0] #already averaged across frequencies
        #con = np.sum([con, con.T], 0)
        con_sym = np.empty(con.shape);
        for fidx in range(con.shape[2]):
            con_sym[:, :, fidx] = np.sum([con[:, :, fidx], con[:, :, fidx].T], 0)
            tmp = con_sym[:, :, fidx]
            tmp[np.diag_indices_from(tmp)] = np.nan
            con_sym[:, :, fidx] = tmp
        con = con_sym

        cohs = []
        for idxi, i in enumerate(hipp_filt):
            if i:
                for idxj, j in enumerate(~hipp_filt):
                    if j:
                        cohs.append(con[idxi, idxj, :])
        cohs = np.array(cohs)

        coh_mean = np.nanmean(logit(cohs), 0)
        event_coh.append(coh_mean)
    
    return np.array(event_coh), fdone, hipp_filt

def get_hipp_phg_plv(reader, evs, mtl_elecs, ref):
    
    #including a 1000 ms buffer because using wavelets
    eeg = reader.load_eeg(events=evs, rel_start=-2000, rel_stop=1000, scheme=mtl_elecs)  #Looking at 1 second prior to retrieval events

    #Now get MNE format for power extraction
    eeg = eeg.to_mne()
    if ref=='mono':
        eeg.set_eeg_reference(ref_channels=None) #Set to average reference
        eeg.apply_proj()
    sr = 500
    eeg = eeg.resample(sr)  #downsample to ___ Hz
    
    #select only hippocampal elecs
    hipp_filt = np.zeros(len(mtl_elecs))
    for idx, i in enumerate(mtl_elecs['stein.region']):
        if np.sum([(s_ in i) for s_ in ['CA', 'DG', 'SUB', 'Sub']])>0:
            hipp_filt[idx] = 1
    hipp_filt = hipp_filt.astype(bool)
    if (np.sum(hipp_filt)==0) | np.sum(hipp_filt)==len(hipp_filt):  #don't run if no hipp/phg contacts
        raise ValueError

    #Use MNE to get phase information
    from mne.time_frequency import tfr_array_morlet
    
    #loop through events and get hipp/phg coherence
    phases = tfr_array_morlet(eeg.get_data(), sfreq=sr, freqs=[4, 5, 6, 7, 8],
                              n_cycles=5.0, output='phase', verbose=False)  #phase output is events x channels x freqs x time
    phases = phases[:, :, :, int(sr*1.0):-1*int(sr*1.0)]
    
    #get plv over time
    from pycircstat import resultant_vector_length, cdiff
    from scipy.special import logit
    
    prs = []
    for idxi, i in enumerate(hipp_filt):
        if i:
            for idxj, j in enumerate(~hipp_filt):
                if j:
                    diff = cdiff(phases[:, idxi, :, :], phases[:, idxj, :, :])
                    prs.append(diff)
    prs = np.array(prs)  #has dimensions pairs, events, freqs, time
    plvs = resultant_vector_length(prs, axis=3)  #has dimensions pairs, events, freqs
    plvs = np.mean(np.mean(plvs, 2), 0)  #has dimensions events 

    return plvs
    

def get_list_power(reader, ev, mtl_elecs, freqs, ref):
    
    eeg = reader.load_eeg(events=ev, rel_start=0, rel_stop=int(30*1000), scheme=mtl_elecs)  #Looking at 1 second prior to retrieval events

    #Now get MNE format for power extraction
    eeg = eeg.to_mne()
    if ref=='mono':
        eeg.set_eeg_reference(ref_channels=None) #Set to average reference
        eeg.apply_proj()
    eeg = eeg.resample(500)  #downsample to ___ Hz

    #Use MNE to get multitaper power in theta and HFA bands
    from mne.time_frequency import psd_multitaper
    theta_pow, fdone = psd_multitaper(eeg, fmin=freqs[0], fmax=freqs[1], tmin=0.0, verbose=False)  #power is (events, elecs, freqs)
    theta_pow = np.mean(np.log10(theta_pow), 1)  #Average across electrodes
    
    #Going to get a lot of needless frequencies, so just grab freqs in 1Hz intervals (every 30 units)
    fdone = np.array(fdone)[::30]
    theta_pow = theta_pow[0, ::30]
    
    return theta_pow, fdone

def get_list_con(reader, evs, mtl_elecs, freqs, ref):
    
    eeg = reader.load_eeg(events=evs, rel_start=0, rel_stop=int(30*1000), scheme=mtl_elecs)  #Looking at 1 second prior to retrieval events

    #Now get MNE format for power extraction
    eeg = eeg.to_mne()
    if ref=='mono':
        eeg.set_eeg_reference(ref_channels=None) #Set to average reference
        eeg.apply_proj()
    sr = 500
    eeg = eeg.resample(sr)  #downsample to ___ Hz
    
    #select only hippocampal elecs
    hipp_filt = np.zeros(len(mtl_elecs))
    for idx, i in enumerate(mtl_elecs['stein.region']):
        if np.sum([(s_ in i) for s_ in ['CA', 'DG', 'SUB', 'Sub']])>0:
            hipp_filt[idx] = 1
    hipp_filt = hipp_filt.astype(bool)
    if (np.sum(hipp_filt)==0) | np.sum(hipp_filt)==len(hipp_filt):  #don't run if no hipp/phg contacts
        raise ValueError

    #Use MNE to get multitaper power in theta and HFA bands
    from mne.connectivity import spectral_connectivity
    from scipy.special import logit
    
    #loop through events and get hipp/phg coherence
    event_coh = []
    for ev in range(len(eeg)):
        con, fdone, times, n_epochs, n_tapers = spectral_connectivity(eeg[ev], method='coh', mode='multitaper', sfreq=sr, fmin=freqs[0], fmax=freqs[1], faverage=False, tmin=0.0, 
                                                                          mt_adaptive=False, verbose=False, mt_bandwidth=None)
        #con = con[:, :, 0] #already averaged across frequencies
        #con = np.sum([con, con.T], 0)
        con_sym = np.empty(con.shape);
        for fidx in range(con.shape[2]):
            con_sym[:, :, fidx] = np.sum([con[:, :, fidx], con[:, :, fidx].T], 0)
            tmp = con_sym[:, :, fidx]
            tmp[np.diag_indices_from(tmp)] = np.nan
            con_sym[:, :, fidx] = tmp
        con = con_sym

        cohs = []
        for idxi, i in enumerate(hipp_filt):
            if i:
                for idxj, j in enumerate(~hipp_filt):
                    if j:
                        cohs.append(con[idxi, idxj, :])
        cohs = np.array(cohs)

        coh_mean = np.nanmean(logit(cohs), 0)
        event_coh.append(coh_mean)
    
    return np.array(event_coh)[0, ::30], fdone[::30], hipp_filt
    

#load word2vec data
model = pk.load(open('/home1/esolo/notebooks/Semantic_dimensions/wordpool_feats.pk', 'rb'))

In [ ]:
import pickle as pk
from sklearn.decomposition import PCA
from scipy.stats import zscore, pearsonr, spearmanr
from scipy.spatial.distance import euclidean
from scipy.stats.mstats import mquantiles
from copy import copy
import pandas as pd

ndim = 11
freq_range = [1, 150]
ref_type = 'bi'
do_conn = False

res = pd.DataFrame()
sub_database = pk.load(open('/scratch/esolo/Semantic_dimensions/subject_database.pkl', 'rb'))
mtl_elecs = []

for s in FR_subs[:]:
    
    #Load subjects information
    exp = 'FR1'
    sessions = df[np.logical_and(df["subject"] == s, df['experiment']==exp)]['session'].unique()

    for sess in sessions:
        
        sess_theta_r = np.empty((ndim-1, 150))  #should not hard-code frequency number, it's numfreqs*window(sec)
        sess_theta_regress = np.empty(sess_theta_r.shape)
        sess_theta_short = np.empty(sess_theta_r.shape)
        sess_theta_long = np.empty(sess_theta_r.shape)
        sess_con_r = np.empty(sess_theta_r.shape)
        sess_con_long = np.empty(sess_theta_long.shape)
        sess_con_short = np.empty(sess_theta_short.shape)
    
        loc = int(df[(df['subject']==s) & (df['session']==sess) & (df['experiment']==exp)]['localization'])
        mont = int(df[(df['subject']==s) & (df['session']==sess) & (df['experiment']==exp)]['montage'])

        #Get task events
        try:
            reader = CMLReader(s, exp, sess, montage=mont, localization=loc)

            mtl_elecs, evs = get_elecs_and_evs(reader, ref_type, ap_thresh=None)
            word_evs = evs[evs['type']=='WORD']
            perf = np.sum(word_evs['recalled'])/float(len(word_evs))

            if len(mtl_elecs)>0:
                pass
            else:
                continue
        except:
            print('Unable to load MTL electrode/word info!')
            continue

        all_theta = []
        all_trans = list([[] for i in range(ndim-1)])
        all_outputs = []   
        all_temp_trans = []
        all_serial_pos = []
        all_con = []

        try:
            new_model, exp_var = get_session_model(model_full, word_evs['item_name'], ndim)
        except:
            continue

        words_done = []  #for storing all the words seen so far
        list_clustering = []
        list_pows = []
        list_con = []
        list_perfs = []
                
        for listnum in word_evs['list'].unique()[:]:
            try:

                #Get info from one list
                words, list_dat = get_list_words(word_evs, listnum)
                words_done.extend(list(words))

                #Get recall events and their semantic values 
                rec_evs = evs[(evs['type']=='REC_WORD') & (evs['list']==listnum) & (evs['intrusion']==0)]  #ignoring intrusions. should address repeat words.
                serial_pos = [int(list_dat[list_dat['item_name']==w]['serialpos'])-1 for w in rec_evs['item_name']]
                serial_pos, repeats_removed = remove_repeats(serial_pos)

                tmp = np.ones(len(rec_evs)).astype(bool); tmp[repeats_removed] = False
                if np.sum(tmp)<4: #threshold for list-level performance
                    continue

                #mask = find_temporal_runs(serial_pos)
                mask = np.zeros(len(serial_pos)).astype(bool)
                mask[0:] = True  #mask is constructed at the word level, not transition level

                #Get percentile rank score for each transition 
                for dimidx, ndim_ in enumerate(range(1, ndim)):

                    #Project semantic features for this list to N dimensions
                    feats = np.array([model_full[w.lower()] for w in words])  #construct feature matrix from one list
                    pca = PCA(n_components=ndim_)
                    pcs = pca.fit_transform(feats)
                    exp_var = pca.explained_variance_ratio_
                    if ndim_==1:
                        positions_1d = copy(pcs)

                    #pcs = np.array([new_model[w] for w in words])[:, :ndim_]

                    positions = pcs
                    pcts = get_recall_clustering(positions, serial_pos) #Small number means short transition, spits out a "closeness" measure -- bigger numbers are closer items

                    all_trans[dimidx].extend(np.array(pcts)[mask[:-1]])
                    if ndim_==1:
                        list_clustering.append(np.nanmean(pcts))

                list_perfs.append(len(serial_pos)/12.)

                #Also get temporal clustering
                temp_coords = np.empty((len(positions), 1))
                temp_coords[:, 0] = np.arange(len(positions))
                temp_pcts = get_recall_clustering(temp_coords, serial_pos)
                all_temp_trans.extend(np.array(temp_pcts)[mask[:-1]])

                #Get pre-recall EEG power in region (1-second prior to recall)

                #Get spectral power (and connectivity)
                theta_pow, fdone = get_power(reader, rec_evs.iloc[tmp], mtl_elecs, freq_range, ref_type)
                #list_pow, fdone_list = get_list_power(reader, evs[(evs['type']=='REC_START') & (evs['list']==listnum)].iloc()[0:1],
                #                                     mtl_elecs, freq_range, ref_type)
                list_pow = []

                #con, fdone, _ = get_list_con(reader, evs[(evs['type']=='REC_START') & (evs['list']==listnum)].iloc()[0:1],
                #                                     mtl_elecs, freq_range, ref_type)
                list_con = []

                list_pows.append(list_pow)
                #list_con.append(con)

                if do_conn:
                    con = get_hipp_phg_plv(reader, rec_evs.iloc[tmp], mtl_elecs, ref_type)
                else:
                    con = []

                #Aggregate data
                all_theta.extend(list(theta_pow[mask, :][1:]))  #use 1: for within-transition theta, or :-1 for before-transition theta
                all_outputs.extend(list(np.arange(0, len(theta_pow))[mask][:-1]))
                all_serial_pos.extend(list(serial_pos[mask][:-1]))
                if do_conn:
                    all_con.extend(list(np.array(con)[mask][:-1]))
                else:
                    all_con.extend([])

#                 print('all_theta: '+str(len(all_theta)))
#                 print('all_outputs: '+str(len(all_outputs)))
#                 print('all_serial_pos: '+str(len(all_serial_pos)))
#                 print('all_temp_trans: '+str(len(all_temp_trans)))
#                 print('all_trans: '+str(len(all_trans[0])))

            except:
                continue

        if len(justfinites(all_trans[0]))<10:  #don't use sessions w/o at least N datapoints for the correlations
            continue

        #Correlate spectral power with transition distances
        try:
            all_theta = np.array(all_theta)
            all_con = np.array(all_con)

            #list-level metric
            list_pows = np.array(list_pows)
            list_clustering = np.array(list_clustering)
            list_con = np.array(list_con)
            list_perfs = np.array(list_perfs)

            theta_avg_r = []; tru_avg_coefs = [];
            con_avg_r = [];

            for dimidx, dim_dat in enumerate(all_trans):  #all dimensions
                for f in range(len(fdone)):  #all frequencies

                    theta_r, theta_p = pearsonr(all_theta[np.isfinite(dim_dat), f], justfinites(dim_dat))
                    sem_longs, sem_shorts = get_long_short(dim_dat, long_thresh=0.25, short_thresh=0.75)
                    theta_short = np.nanmean( zscore(all_theta[:, f])[sem_shorts] )
                    theta_long = np.nanmean( zscore(all_theta[:, f])[sem_longs] )
                    if do_conn:
                        if len(con.shape)==1:
                            con_r, con_p = pearsonr(all_con[np.isfinite(dim_dat)], justfinites(dim_dat))
                            con_short = np.nanmean( all_con[:][sem_shorts] )
                            con_long = np.nanmean( all_con[:][sem_longs] ) 
                        else:
                            con_r, con_p = pearsonr(all_con[np.isfinite(dim_dat), f], justfinites(dim_dat))
                            con_short = np.nanmean( all_con[:, f][sem_shorts] )
                            con_long = np.nanmean( all_con[:, f][sem_longs] )   #connectivity metrics already normalized
                    else:
                        con_r = np.nan
                        con_short = np.nan
                        con_long = np.nan

                    tru_coefs = regress_output_pos(all_theta[np.isfinite(dim_dat), f], dim_dat, all_outputs, all_serial_pos)

                    sess_theta_regress[dimidx, f] = tru_coefs[0]
                    sess_theta_r[dimidx, f] = theta_r
                    sess_theta_short[dimidx, f] = theta_short
                    sess_theta_long[dimidx, f] = theta_long
                    sess_con_r[dimidx, f] = con_r
                    sess_con_short[dimidx, f] = con_short
                    sess_con_long[dimidx, f] = con_long

        except:
            continue  #usually an EEG reading error, or not enough transitions
        
        try:
            #Temporal clustering X frequency
            temp_by_freq = []
            temp_regress = []; temp_long = []; temp_short = []
            con_temp = []; con_temp_long = []; con_temp_short = []
            for f in range(len(fdone)):
                temp_r, temp_p = pearsonr(all_theta[np.isfinite(all_temp_trans), f], justfinites(all_temp_trans))
                temp_by_freq.append(temp_r)

                if do_conn:
                    if len(con.shape)==1:
                        con_temp_r, _ = pearsonr(all_con[np.isfinite(all_temp_trans)], justfinites(all_temp_trans))
                        con_temp.append(con_temp_r)
                    else:
                        con_temp_r, _ = pearsonr(all_con[np.isfinite(all_temp_trans), f], justfinites(all_temp_trans))
                        con_temp.append(con_temp_r)
                else:
                    con_temp.append(np.nan)

                coefs_temp = regress_output_pos(all_theta[np.isfinite(all_temp_trans), f], justfinites(all_temp_trans), all_outputs, all_serial_pos)
                temp_regress.append(coefs_temp[0])

                temp_longs, temp_shorts = get_long_short(all_temp_trans, long_thresh=0.25, short_thresh=0.75)
                temp_long.append( np.nanmean( zscore(all_theta[:, f])[temp_longs] )  )
                temp_short.append( np.nanmean( zscore(all_theta[:, f])[temp_shorts] )  )

                if do_conn:
                    if len(con.shape)==1:
                        con_temp_long.append( np.nanmean( all_con[:][temp_longs] )   )
                        con_temp_short.append(   np.nanmean( all_con[:][temp_shorts] ) )
                    else:
                        con_temp_long.append( np.nanmean( all_con[:, f][temp_longs] )   )
                        con_temp_short.append(   np.nanmean( all_con[:, f][temp_shorts] ) )
                else:
                    con_temp_long.append(np.nan)
                    con_temp_short.append(np.nan)


            temp_by_freq = np.array(temp_by_freq)
            con_temp = np.array(con_temp)
            temp_regress = np.array(temp_regress)
            temp_long = np.array(temp_long); temp_short = np.array(temp_short)
            con_temp_long = np.array(con_temp_long)
            con_temp_short = np.array(con_temp_short)
        except:
            continue
        
        sem_Z = sub_database[sub_database['subject']==s]['sem_Z']
        tem_Z = sub_database[sub_database['subject']==s]['tem_Z']
        res = res.append({'subject': s, 'localization': loc, 'montage': mont, 'session':sess,
                        'r':sess_theta_r, 'r_regress':sess_theta_regress, 'power_short':sess_theta_short,
                        'power_long':sess_theta_long, 'frequencies':fdone, 'dimensions':list(range(1, ndim)), 
                         'performance':perf, 'exp_var':exp_var, 'temporal_clustering_r': temp_by_freq,
                         'semantic_transitions': all_trans[0], 'temporal_transitions': all_temp_trans,
                         'temp_regress': temp_regress, 'temp_short': temp_short, 'temp_long': temp_long,
                         'sem_Z':float(sem_Z), 'tem_Z':float(tem_Z), 'con_r': sess_con_r, 'con_sem_long': sess_con_long,
                         'con_sem_short': sess_con_short, 'con_tem_long': con_temp_long, 'con_tem_short': con_temp_short, 'con_temp_r': con_temp,
                         'list_pow':list_pows, 'list_clustering':list_clustering, 'list_con':list_con, 'list_perfs':list_perfs}, ignore_index=True)
        

    print(s)
    
